In [1]:
import os, sys
import shutil
import pandas as pd
import numpy as np

In [2]:
# !jt -l
# # !jt -T
# !jt -r

In [3]:
main = os.getcwd()
print('main: ', main)
os.chdir(main + '\\' + 'tests_ignore')
prj = os.getcwd()
print('prj: ', prj)
# os.chdir(main)
# os.getcwd()

main:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad
prj:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore


In [4]:
# for root, dirs, files in os.walk(prj):
#     print(root)
#     print(dirs)
#     print(files)

In [5]:
shutil.copy2('ECG_MOUSE.csv', 'test_copy')
for root, dirs, files in os.walk(prj):
    print(files)

['1217861-1.dcm', 'ECG_MOUSE-cache.lib', 'ECG_MOUSE.csv', 'ECG_MOUSE.lib', 'ECG_MOUSE.net', 'main_copy.lib', 'orig_ECG_MOUSE', 'templibrary.lib', 'test_copy']


## Extract .csv data from file into DataFrame

In [6]:
mpn = "Manufacturer Part Number"
mpn_ = 'Manufacturer_Part_Number'
mpno = 'Manufacturer Part No.'
manuf = 'Manufacturer'
manuf_ = 'Manufacturer_Name'
ampn = 'Alt. Manufacturer Part Number'
amanuf = 'Alt. Manufacturer'
ref = 'Reference'
ds = 'Datasheet'
val = 'Value'
fp = 'Footprint'

with open('test_copy', 'r') as schemtxt:
    data = schemtxt.read().splitlines()
    headers = data[0].split(', ')       #headers need to be labeled headers
    for line in range(0, len(data)):
        # data[line] = data[line].split('","')
            #replace commas with double => prev. issues @mpns w/commas
        data[line] = data[line].replace('","', ',,').strip('"')
        # data[line] = data[line][:-1]
        data[line] = data[line].split(',,')
        
    data = data[1:len(data)]
    #re-init data w/o headers

    df = pd.DataFrame(data, index = np.arange(1, len(data)+1), columns = headers)
#     df[mpn_] = df[mpn_].str.strip('"')
#     df[ref] = df[ref].str.strip('"')
#     df[ds] = df[ds].str.strip('"')

    print(df.head(5))


Reference  Value                                          Footprint  \
1       TP1   5019  Measurement_Points:Test_Point_Keystone_5019_Mi...   
2       C20    .1u                              Capacitors_SMD:C_0805   
3       C18    47u                     Capacitors_SMD:CP_Elec_6.3x7.7   
4        D2  GREEN                                      LEDs:LED_0603   
5        D1  GREEN                                      LEDs:LED_0603   

                                           Datasheet          Manufacturer  \
1      http://www.keyelco.com/product-pdf.cfm?p=1357  Keystone Electronics   
2  https://content.kemet.com/datasheets/KEM_C1002...                 Kemet   
3  https://industrial.panasonic.com/ww/products/c...             Panasonic   
4  https://media.digikey.com/pdf/Data%20Sheets/Li...                LITEON   
5  https://media.digikey.com/pdf/Data%20Sheets/Li...                LITEON   

        Part Number Manufacturer Part No. Description Manufacturer_Name  \
1              5019

## Clean Dataframe Content

In [7]:
"""
columns already strings => str.cat works (string concatenation)
https://stackoverflow.com/questions/19377969/combine-two-columns-of-text-in-dataframe-in-pandas-python 
    """
# df = df.rename(columns = {'Part Number' : 'MPN', 
#                             'Alternative Part Number' : 'Alt. MPN',
#                             'Alternative Manufacturer' : 'Alt. Manufacturer'})
################################## PART NUMBERS ##################################

            #combine + delete part no. columns
# if (len(df.columns.values[5]) == 0):

if mpno or mpn_ in df.columns.values:
    df['Part Number'] = df['Part Number'].str.cat(df[[mpno, mpn_]])
    df = df.drop([mpn_, mpno], axis=1)
    df = df.rename(columns = {'Part Number' : mpn})

        #drops the column - df.drop([x1, x2]) allows multiple vs df.pop() and del

################################## MANUF NAME ##################################
#repeat of part numbers
# if (len(df.columns.values[4]) == 0):

if manuf_ in df.columns.values:
    df[manuf] = df[manuf].str.cat(df[manuf_])
    df = df.drop([manuf_], axis=1)

df = df.drop(['Description'], axis=1)
df[amanuf] = ''
df[ampn] = ''


In [8]:
# df[amanuf] = ''
# df[ampn] = ''
# headers = df.columns.values.tolist()
# print(headers)
# print()
# # print(type(df[ampn]))
# for item in df.itertuples():
#     print(type(item))
#     print(item)
#     # u = item.tolist()
#     print(type(u), '\n', u)

#     break;

#### At this point, the escheema file with no extension (the un-extensioned .csv file) that contains the schematic listed information for all of the parts on the schematic has been...
* converted into a dataframe - BINGO
  * move on to library
* had extra fields removed - BINGO 
  * move on to library
* successfully interpretable by KiCad - NOPE
  * STOP PURSUIT

Turns out there was an error in how I was running the command line operation for the scripts to convert a schematic into a .csv file. We noticed that we received a .xml file, but after some further research and the __failure of the third bullet__, I discovered that the file with no extension... Was in fact... the .csv file that we were missing. 

* This has now been resolved by adding the needed .csv after the capital O in the command line script. 

In [9]:
# print(headers)
# print(type(df))
# print(df.shape, df.size)
# print(df)

____
## Creating the Library (From .csv)

In [75]:
import os, sys
import shutil
import pandas as pd
import numpy as np


In [76]:
# %%javascript
# IPython.OutputArea.auto_scroll_threshold = 9999;

In [77]:
# os.chdir(main)
# os.getcwd()

In [78]:
# main = os.getcwd()
# print('main: ', main)
# os.chdir(main + '\\' + 'tests_ignore')
prj = os.getcwd()
print('prj: ', prj)

prj:  c:\Users\jgoss\Documents\_Olin\DRF_engineering\LibGen_KiCad\tests_ignore


In [79]:
with open('ECG_MOUSE-cache.lib', 'r') as raw:
    raw = raw.read().split('ENDDEF\n')
    compdetails = dict()
    for item in raw:
        if 'R-Device' in item:                                      #resistor
            compdetails.update({'res' : item + 'ENDDEF\n\n'})
        
        if 'C-Device' in item:                                      #capacitor
            compdetails.update({'c' : item + 'ENDDEF\n\n'})

        if 'CP-Device' in item:                                     #polarized capacitor
            compdetails.update({'cp' : item + 'ENDDEF\n\n'})

#extra types
        # if 'C_Small' in item:
        #     compdetails.update({'csmall' : item})            
            #draw unpolarized cap
#         if 'CP_Small' in item:
#             compdetails.update({'cpsmall' : item})
#             ##draw small unpolarized cap
#         if 'CP1-Device' in item:
# #             cpdraw_us = item
#             compdetails.update({'cp1' : item})
#             #draw us symbol capacitor
#         if 'CP1_Small' in item:
#             compdetails.update({'cp1small' : item})


        item = item + 'ENDDEF\n'+'\n'
# compdetails.update('C0805fp' : 'Capacitors_SMD:C_0805')
# compdetails.update('R0805fp' : 'Resistors_SMD:R_0805')
        #SAAD DIDN'T GIVE ME FOOTPRINT ACCESS
    

Library content was successfully taken out and put into a dictionary containing all of the parts details. Now each dictionary entry needs to be replicated and do the desired thing...

* Create a passive component (Resistor/Capacitor) with the desired symbol
* Assign it a desired value
* Insert the other requested fields
Then a successful library has been generated for import (if header and footer text of file are included)

Cool things to do, that would be cool to accelerate this process
* A list of desired 0805 resistor/capacitor values for sweeping
* A list of desired 0605 resistor/capacitor values for sweeping
* The footprint strings that are necessary
    * The managing of libraries would have to be done as well
* Maybe make a webscraping thing to get these links off of digikey for me?
    * Maybe have like a preferred manufacturer?
    * Manufacturer & MPN + Alts?
* Have a setup that can let me sweep through Saad's other projects as well?
    * Full python script that can run in terminal?
    * Institute classes and other stuff
    * Ask for string input for document sweeping and generation?
    * Learn how to implement a GUI?

In [80]:
def add_fields (manuf, mpn, amanuf, ampn, pck):
    arbpos = ' 0 0 50 H I C CNN'
    field4 = 'F4 "{k}" {j}'.format(k=manuf, j=arbpos)
    field5 = 'F5 "{k}" {j}'.format(k=mpn, j=arbpos)
    field6 = 'F8 "{k}" {j}'.format(k=amanuf, j=arbpos)
    field7 = 'F7 "{k}" {j}'.format(k=ampn, j=arbpos)

    return field4, field5, field6, field7


def libadd (name, prt, pck, field0, field1, field2, field3, field4, field5, field6, field7):
# def libadd (name, prt, pck, *args, **kwargs):
    """
        name = definition
        prt = R/C (resistor or capacitor or polarized capacitor)
        pck = 0805/0603 (package)
        f0 = reference
        f1 = name/definition
        f2 = footprint
        f3 = datasheet
        f4 = manufacturer
        f5 = mpn
        f6 = alt manufacturer
        f7 alt mpn
        """
    
    # for a in range(len(*args)):
    #     a = 'a{f}'.format

    partheader = '#\n# {k}\n#'.format(k=name)
    if prt == 'R':        #regular resistor
        DEF = 'DEF {k} R 0 0 N Y 1 F N'.format(k=name)
        f0 = 'F0 "{k}" 80 0 50 V V C CNN'.format(k=field0)
        f1 = 'F1 "{k}" 80 0 50 V V C CNN'.format(k=field1)
        f2 = 'F2 "{k}" 80 0 50 V V C CNN'.format(k=field2)
        f3 = 'F3 "{k}" 0 0 50 H I C CNN'.format(k=field3)

####### add in the other fields, and footer(footprint + drawing) ####
####### make this part optional """"""""""
        f4, f5, f6, f7 = add_fields(field4, field5, field6, field7, pck)
        # partfooter = compdetails['res'][compdetails['res'].find('$FPLIST'): \
        #                     compdetails['res'].find('\n #')]
        index1 = compdetails['cp'].find('$ENDFPLIST')
        index2 = compdetails['cp'].find('\n #')
        fplst = '$FPLIST\n' + ' {k}_{j}*'.format(k=prt, j=pck)

        partfooter = '\n'.join([fplst, compdetails['cp'][index1:index2]])
        
    if prt == 'C':        #regular capacitor
        DEF = 'DEF {k} C 0 10 N Y 0 F N'.format(k=name)
        f0 = 'F0 "{k}" 25 100 50 H V L CNN'.format(k=field0)
        f1 = 'F1 "{k}" 25 -100 50 H V L CNN'.format(k=field1) 
        f2 = 'F2 "{k}" 38 -150 50 H I C CNN'.format(k=field2)
        f3 = 'F3 "{k}" 0 0 50 H I C CNN'.format(k=field3)

        f4, f5, f6, f7 = add_fields(field4, field5, field6, field7, pck)
        index1 = compdetails['cp'].find('$ENDFPLIST')
        index2 = compdetails['cp'].find('\n #')
        fplst = '$FPLIST\n' + ' {k}_{j}*'.format(k=prt, j=pck)

        partfooter = '\n'.join([fplst, compdetails['cp'][index1:index2]])        
        # partfooter = compdetails['c'][compdetails['c'].find('$FPLIST'): \
        #                     compdetails['c'].find('\n #')]

    if prt == 'CP':           #polarized capacitor
        DEF = 'DEF {k} C 0 10 N Y 1 F N'.format(k=name)
        f0 = 'F0 "{k}" 25 100 50 H V L CNN'.format(k=field0)
        f1 = 'F1 "{k}" 25 -100 50 H V L CNN'.format(k=field1)
        f2 = 'F2 "{k}" 38 -150 50 H I C CNN'.format(k=field2)
        f3 = 'F3 "{k}" 0 0 50 H I C CNN'.format(k=field3)
        
        f4, f5, f6, f7 = add_fields(field4, field5, field6, field7, pck)
        index1 = compdetails['cp'].find('$ENDFPLIST')
        index2 = compdetails['cp'].find('\n #')
        fplst = '$FPLIST\n' + ' {k}_{j}*'.format(k=prt, j=pck)

        partfooter = '\n'.join([fplst, compdetails['cp'][index1:index2]])+'\n'

    newprt = '\n'.join([partheader, DEF, f0, f1, f2, \
                f3, f4, f5, f6, f7, partfooter])

        
    return newprt


test = libadd('name', 'C', '0805', 'thing0', 'thing1', 'thing2', 'thing3', 'thing4', 'thing5', 'thing6', 'thing7')
print(test)

#
# name
#
DEF name C 0 10 N Y 0 F N
F0 "thing0" 25 100 50 H V L CNN
F1 "thing1" 25 -100 50 H V L CNN
F2 "thing2" 38 -150 50 H I C CNN
F3 "thing3" 0 0 50 H I C CNN
F4 "thing4"  0 0 50 H I C CNN
F5 "thing5"  0 0 50 H I C CNN
F8 "thing6"  0 0 50 H I C CNN
F7 "thing7"  0 0 50 H I C CNN
$FPLIST
 C_0805*
$ENDFPLIST
DRAW
S -90 20 90 40 0 1 0 N
S 90 -20 -90 -40 0 1 0 F
P 2 0 1 0 -70 90 -30 90 N
P 2 0 1 0 -50 110 -50 70 N
X ~ 1 0 150 110 D 50 50 1 1 P
X ~ 2 0 -150 110 U 50 50 1 1 P
ENDDRAW
ENDDEF



In [81]:
"""
    name = definition
    prt = R/C (resistor or capacitor or polarized capacitor)
    pck = 0805/0603 (package)
    f0 = reference
    f1 = name/definition
    f2 = footprint
    f3 = datasheet
    f4 = manufacturer
    f5 = mpn
    f6 = alt manufacturer
    f7 = alt mpn
    """

mpn = '_6'
amanuf = '_7'
ampn = '_8'
val = "Value"


with open('templibrary.lib', 'w') as libby:
    with open('ECG_MOUSE.lib', 'r') as ecg:
        initlines = ecg.readlines()
        print(initlines)
        libby.write(''.join(initlines[0:2]))
    for part in df.itertuples():
        # print(part)
        name = part.__getattribute__(val)
        f0 = part.__getattribute__(val)
        f1 = part.__getattribute__(ref)
        ptype = f1[0]
        f2 = part.__getattribute__(fp)
        f3 = part.__getattribute__(ds)
        f4 = part.__getattribute__(manuf)
        f5 = part.__getattribute__(mpn)
        f6 = part.__getattribute__(amanuf)
        f7 = part.__getattribute__(ampn)
        # print('hello', f5, f6, f7)
        # print(name, f0, f1, f2, f3, f4, f5)
        if '0805' in part.__getattribute__(fp):
            pck = '0805'
        elif '0805' in part.__getattribute__(fp):
            pck = '0805'

        if 'R' in f1[0] and 'RV' not in f1:
            libby.write(libadd(name, ptype, pck, f0, f1, f2, f3, f4, f5, f6, f7))
        if 'C' in f1[0]:
            libby.write(libadd(name, ptype, pck, f0, f1, f2, f3, f4, f5, f6, f7))
    libby.write('\n#\nEND Library')

['EESchema-LIBRARY Version 2.4\n', '#encoding utf-8\n', '#\n', '# +5V-power\n', '#\n', 'DEF +5V-power #PWR 0 0 Y Y 1 F P\n', 'F0 "#PWR" 0 -150 50 H I C CNN\n', 'F1 "+5V-power" 0 140 50 H V C CNN\n', 'F2 "" 0 0 50 H I C CNN\n', 'F3 "" 0 0 50 H I C CNN\n', 'DRAW\n', 'P 2 0 1 0 -30 50 0 100 N\n', 'P 2 0 1 0 0 0 0 100 N\n', 'P 2 0 1 0 0 100 30 50 N\n', 'X +5V 1 0 0 0 U 50 50 1 1 W N\n', 'ENDDRAW\n', 'ENDDEF\n', '#\n', '# -5V-power\n', '#\n', 'DEF -5V-power #PWR 0 0 Y Y 1 F P\n', 'F0 "#PWR" 0 100 50 H I C CNN\n', 'F1 "-5V-power" 0 150 50 H V C CNN\n', 'F2 "" 0 0 50 H I C CNN\n', 'F3 "" 0 0 50 H I C CNN\n', 'DRAW\n', 'P 6 0 1 0 0 0 0 50 30 50 0 100 -30 50 0 50 F\n', 'X -5V 1 0 0 0 U 50 50 0 0 W N\n', 'ENDDRAW\n', 'ENDDEF\n', '#\n', '# AD8421\n', '#\n', 'DEF AD8421 U 0 20 Y Y 1 F N\n', 'F0 "U" 150 300 50 H V C CNN\n', 'F1 "AD8421" 150 200 50 H V C CNN\n', 'F2 "" -300 0 50 H I C CNN\n', 'F3 "" 350 -400 50 H I C CNN\n', '$FPLIST\n', ' SOIC*3.9x4.9mm*Pitch1.27mm*\n', ' MSOP*3x3mm*Pitch0.65mm*\n'

In [82]:
for thing in df.itertuples():
    print(thing)
    print(type(thing), '\n')

    print(thing[2])
    print(type(thing[2]), '\n')

    print('ref is ' + thing.__getattribute__(ref))
    x = thing.__getattribute__(ref)
    print(type(x), 'type of x = ' + x)
    print('\n')
    # print(type(df[manuf]))
    # print(df[manuf])

    print(df.columns.values)
    # print(df.columns.values)

    break;
# with open('ECG_MOUSE.lib', 'r') as ecg:
#     initlines = ecg.readlines()
#     print(initlines[0:2])
# print(df.__getattribute__('Manufacturer'))

Pandas(Index=1, Reference='TP1', Value='5019', Footprint='Measurement_Points:Test_Point_Keystone_5019_Minature', Datasheet='http://www.keyelco.com/product-pdf.cfm?p=1357', Manufacturer='Keystone Electronics', _6='5019', _7='', _8='')
<class 'pandas.core.frame.Pandas'> 

5019
<class 'str'> 

ref is TP1
<class 'str'> type of x = TP1


['Reference' 'Value' 'Footprint' 'Datasheet' 'Manufacturer'
 'Manufacturer Part Number' 'Alt. Manufacturer'
 'Alt. Manufacturer Part Number']


In [29]:
df.columns[5]

'Manufacturer Part Number'

__________________
<br/><br/><br/><br/><br/><br/><br/><br/><br/><br/><br/>






___________

_______
## Extracting from the Netlist

In [ ]:
with open('ECG_MOUSE.net', 'r') as netlist:
    temp = netlist.read()
    startcomp, endcomp = temp.find('(component'), temp.find('(libparts')
    # print(type(startcomp), startcomp, endcomp)
    # print(temp[endcomp])
    temp = temp[startcomp:endcomp]
#     print(temp)
            # the components section has been extracted
    with open('netcomp.lib', 'w') as nc: 
            #netlist of components == netcomp
        nc.write(temp)
        

In [ ]:
# ha = 'abcdefghijklmnop\n' + \
        # 'qrstuvwxyz\n' + 'button! up'
# print(ha.find('j', end=ha.find('button!')))
# ha.find('button!')
thing = 'thing'
print(type(thing[0]))

In [ ]:
# print(df)